# Propagation of Sound in Open Water

In [How do acoustics signals look like?]((acoustics-receiver_time_series)), we talked about a sound wave and considered what the sound would look like if you were measuring it at a single point using a hydrophone. This measured sound is only a small part of the much larger distribution of sound traveling through the water. It might be coming from a single source, many sources, or from reflection and scattering from surfaces and objects in the medium. The sound field can become complicated very quickly if we take into account all of the sound moving through the ocean. Instead, let's start by considering a couple simple sound waves.



## The plane wave

Probably the simplest form of sound propagation is the **plane wave**. This is a situation where (1) the sound at every point in the water is traveling in the same direction and (2) the sound has the same amplitude in planes that are perpendicular to the direction of propagation.  The animation below shows the pressure in the x-y plane for a propagating plane wave where the sound is traveling in the x-direction. For this particular case, the wavelength of the wave is 2 m and the amplitude has been chosen to be 1 $\mu Pa$. Notice that in this cross-section of the field, there are lines of constant pressure extending in the y-direction. These lines of constant pressure correspond to lines of constant phase in the wave and these lines are usually referred to as **wave fronts.** Notice that the sound is propagating in the direction perpendicular to the wavefront.

```{image} ../images/PlaneWave.gif
:width: 400px
```

(acoustics-spherical_wave)=
## The spherical wave

Another simple sound wave is the **spherical wave**. For this type of wave, the sound is propagateing in all directions away from a single point. The animation below shows the pressure in the x-y plane for a spherical wave centered on $x = y = 0$. In this case, the wavefronts form circles about the origin and in three dimensions they form spheres. Unlike the plane wave, the spreading of the sound from a single point leads to a decrease in amplitude as you move away from the center. The magnitude of the pressure is inversely proportional to the distance, $r$, from the center, $P \propto 1/r$. The intensity of the sound is then proportional to the square of the distance, $I \propto P^2 \propto 1/r^2$. Also notice that the sound is again propagating in a direction perpendicular to the wavefront. In fact, this is a key point about propagating sound: **Sound always travels in the direction perpendicular to the wavefront.** We will revisit some of the consequences of this property when we examine [acoustic refraction](acoustics-refraction).

```{image} ../images/SphericalWave.gif
:width: 400px
```

Spherical waves are typically generated using a [spherical source](acoustics-spherical_source). In the animation, a small black circle has been placed in the center to denote the location of the source. In the widget later in this notebook and in subsequent notebooks, we will consider a special case of the spherical source called the **point source**. This is an artificial spherical source, useful for modeling and simulation, where the radius of source has been allowed to decrease to zero without affecting the level of the transmitted sound.  In the remainder of this section, we will use this simple source to explore what happens to the sound it produces when we introduce other simple sources, changes in the properties of the water, or boundaries such as the sea surface or seafloor. To keep things simple, we will assume a [source level]((acoustics-source_level)), $SL$, of 0 dB, which means that at 1 m from the source, the magnitude of the pressure will be about 1.4 $\mu Pa$.

```{Note}
:class: note
Why consider an unrealistic point source instead of a more realistic spherical source? As you move closer to a transducer you enter into the [near-field of the transducer]((acoustics-near_field)) where the sound from different parts of the transducer surface interfere created a complicated sound field. The spherical source also has a near field, the boundary of which is proportional to both the frequency of the transmitted sound and the radius of the source. By reducing that radius to a point, we eliminate the near field. Thus the point source is a useful, but artificial sound source.
``` 


## Interference of sound

Before we look at how the underwater environment affects the sound from our point source, we will begin by considering how sound interacts with sound. This interaction of sound with sound can be caused by multiple sources (as we will examine in this notebook) or by the environment reflecting or refracting the sound from a single source, causing it to interact with itself. When sound interacts with sound, two things happen:

1. The propagating sound doesn't change it's path or properties.
2. Sound propagating from different directions and/or with different frequencies will interfere with one another at the points where their paths intersect.

To see this, will use a simple widget that simulates the sound emitted by either one or two point sources. When source #2 is off, the plot shows something very similar to the animation above for a spherical wave, but only a single instance of time. When the second source is turned on and the plot is set to show pressure, the circular wavefronts remain, but now there is **interference** between the two sound fields. Interference occurs because at each point the total pressure is the sum of the pressures from the the two sources. If the pressures are roughly the same (either both have positive or negative values), the addition leads to a larger pressure magnitude. This is called **constructive interference**. When the pressures have opposite signs, the two pressures cancel which leads to **destructive interference.** This is more clearly seen in the intensity plot where the destructive interference corresponds to the lines of low intensity which pass between the two sources. 

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Set inline
%matplotlib inline

In [2]:
def two_sources_field(source_2, plot_type, frequency, distance_apart):
    """
    Plot two sources field.
    """
    # Speed of sound (m/s)
    c = 1500.0
    
    # Pressure amplitudes
    SL = 0
    A_SL = 10**(SL/20)

    # Turn on or off second source
    A2 = 1 if source_2 == "On" else 0

    # Maximum seperation between sources
    dmax = 10

    # Compute domain of calculation
    Lx = 2 * dmax
    Ly = 2 * dmax
    Nx = 150
    Ny = Nx
    x = np.linspace(-Lx, Lx, Nx)
    y = np.linspace(-Ly, Ly, Ny)
    X, Y = np.meshgrid(x, y)

    # Determine pressure field from each source and sum

    # Distance from source #1 to each point
    r1 = np.sqrt((X - distance_apart/2)**2 + Y**2)

    # Distance from source #2 to each point
    r2 = np.sqrt((X + distance_apart/2)**2 + Y**2)

    # Pressure due to source #1
    p1 = A_SL * np.exp(1j * 2*np.pi*frequency*r1/c) / r1

    # Pressure due to source #2
    p2 = A_SL * A2 * np.exp(1j * 2*np.pi*frequency*r2/c) / r2

    # Coherent sum of pressures
    p = p1 + p2

    # Create plot for either pressure or intensity acoustic fields
    fig, ax = plt.subplots(figsize=(8, 6))
    if plot_type == "Pressure":
        pressure_data = np.real(p) * np.sqrt(2)
        im = ax.imshow(pressure_data, extent=[x[0], x[-1], y[0], y[-1]], origin="lower", cmap="viridis", vmin=-2, vmax=2)
        ax.set_xlabel("x (m)")
        ax.set_ylabel("y (m)")
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label("Pressure (µPa)")
    else:
        intensity_data = 20 * np.log10(np.abs(p))
        im = ax.imshow(intensity_data, extent=[x[0], x[-1], y[0], y[-1]], origin="lower", cmap="viridis", vmin=-40, vmax=5)
        ax.set_xlabel("x (m)")
        ax.set_ylabel("y (m)")
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label("Intensity (dB re 1 µPa)")
    ax.set_title("Acoustic Field")

    # Show plot
    plt.show()

# Create the interactive widgets
layout = widgets.Layout(width="500px", height="30px")
_ = widgets.interact(
    two_sources_field,
    source_2=widgets.Dropdown(
        options=["On", "Off"],
        value="On",
        description="Source 2",
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    plot_type=widgets.Dropdown(
        options=["Pressure", "Intensity"],
        value="Intensity",
        description="Plot Type",
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    frequency=widgets.FloatSlider(
        value=1200.0,
        min=600.0,
        max=1800.0,
        step=5.0,
        description="Frequency (Hz)",
        style={"description_width": "initial"}, 
        layout=layout,
    ),
    distance_apart=widgets.FloatSlider(
        value=5.5,
        min=0,
        max=20,
        step=0.1,
        description="Distance Apart (m)",
        style={"description_width": "initial"}, 
        layout=layout,
    )
)

interactive(children=(Dropdown(description='Source 2', layout=Layout(height='30px', width='500px'), options=('…